### Preparing Inputs for LLMs
We’ve completed preparing the input to be fed into large language models (LLMs).

### Attention Mechanisms
Now, let’s explore the **attention mechanism** in detail.

We will implement four types of attention mechanisms:
1. **Simplified self-attention**
2. **Self-attention (with trainable weights)**
3. **Causal attention**
4. **Multi-head attention**


### Section 3.1 (The problem with modelling long sequences)

### The Problem with Pre-LLM Architectures Without Self-Attention  

Before the introduction of self-attention, **Recurrent Neural Networks (RNNs)** were the state-of-the-art (SOTA) for tasks like language translation.  

In RNN-based architectures:  
- The encoder processes the entire input sequence into a final hidden state (along with intermediate hidden states after each input word).  
- The decoder uses this final hidden state to generate the output sequence.  

**Key limitation:** The decoder primarily relies on the final hidden state of the encoder, ignoring intermediate hidden states. This often results in a **loss of context**, especially for longer input sequences, as critical information from earlier parts of the input may not be


### Section 3.2 (Capturing data dependencies with attention mechanisms)

### Evolution of Attention Mechanisms  

An improvement over the basic RNN encoder-decoder architecture was the **Bahdanau attention mechanism** (2014), which allowed the decoder to selectively access different parts of the input sequence.  

Three years later, the paper *"Attention Is All You Need"* (2017) introduced the **Transformer architecture**, a significant improvement over the 2014 model.  


### Section 3.3 (Attending to different parts of the input with self-attention)

### Understanding Self-Attention  

#### What is "self" in self-attention?  
"Self" refers to the model's ability to learn relationships and dependencies between various parts of the input (e.g., words in a sentence).  
This is in contrast to traditional attention models, which focus on relationships between two different sequences (e.g., in sequence-to-sequence models).  

#### What is the goal of self-attention?  
The goal of self-attention is to compute a **context vector** for every token in the input sequence.  

#### What is a context vector?  
A **context vector** is generated for each input element by combining information from other input elements. Each element contributes to the context vector based on its **attention score**.  
$$ \text{Context Vector} = \text{Attention Weights} \times \text{Token Embeddings} $$

#### How is the attention score for a token computed?  
The attention score for a token `t` in the input sentence is computed by performing the **dot product** of that token's representation (called the **query**) with the representations of all other tokens in the input sequence.  


#### 1. Simplified self-attention implementation

Below is an example sentence. Each token in the sentence has an embedding. Below are steps to obtain query vector for a token:

In [1]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

/Users/bhavtosh/env1/lib/python3.9/site-packages/torch/nn/modules/transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


Step 1: We compute the attention scores for token 'journey' (the query) by computing its dot product with all tokens in the input sequence.

In [2]:
query = inputs[1]  # 'journey' is the query

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # dot product (transpose not necessary here since they are 1-dim vectors)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


Step 2: We normalize the attention scores using 'softmax' for interpretability and to maintain training stability.

In [3]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


Step 3: Now we compute context vector of token 'journey' by multiplying attention score with corresponding token embeddings of all tokens in the input sentence.


In [4]:
query = inputs[1] # 2nd input token is the query

context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


Now we compute attention weights and context vectors for all inputs

In [5]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


We normalize attention scores

In [6]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


Calculate context vectors


In [7]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


### Section 3.4 (Implementing self-attention with trainable weights)

#### 2. Self-attention (with trainable weights) implementation

We introduce 3 trainable weight matrices (Wq, Wk, Wv). These project the input tokens into query, key and value vectors.

$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right)V
$

Where:  
- \( Q \): Query matrix  
- \( K \): Key matrix  
- \( V \): Value matrix  
- \( d_k \): Dimensionality of the key vectors  

In [10]:
x = inputs[1] # second input element
d_in = inputs.shape[1] # the input embedding size, d=3
d_out = 2 # the output embedding size, d=2

### Initializing Weight Matrices: \( ${W_q}$, ${W_k}$, ${W_q}$ \)

When initializing weight matrices, the value of `requires_grad` determines whether gradients are computed during backpropagation. Here's the distinction:

- **`requires_grad=False`**  
  - Gradients for this tensor will **not** be computed or stored during backpropagation.  
  - Commonly used during **inference**, when gradient tracking is unnecessary.

- **`requires_grad=True`**  
  - Enables gradient computation for the tensor.  
  - All operations on this tensor are tracked, allowing gradients to propagate back during backpropagation.  
  - Typically used during **training**, where gradient updates are required.

For simplicity purpose we keep it False, for now.

In [12]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

Compute the query ${Q}$, key ${K}$ and value ${V}$ vectors

In [22]:
query = inputs @ W_query # _2 because it's with respect to the 2nd input element
key = inputs @ W_key 
value = inputs @ W_value

print(query)

tensor([[0.2309, 1.0966],
        [0.4306, 1.4551],
        [0.4300, 1.4343],
        [0.2355, 0.7990],
        [0.2983, 0.6565],
        [0.2568, 1.0533]])


**STEP 1:** Compute attention scores (unnormalized) [ dot product between the query and each key vector]  
${QK^\top}$

In [23]:
attn_scores = query @ keys.T # All attention scores for given query
print(attn_scores)

tensor([[0.9231, 1.3545, 1.3241, 0.7910, 0.4032, 1.1330],
        [1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
        [1.2544, 1.8284, 1.7877, 1.0654, 0.5508, 1.5238],
        [0.6973, 1.0167, 0.9941, 0.5925, 0.3061, 0.8475],
        [0.6114, 0.8819, 0.8626, 0.5121, 0.2707, 0.7307],
        [0.8995, 1.3165, 1.2871, 0.7682, 0.3937, 1.0996]])


**STEP 2:** Compute attention weights (normalized attention scores).  
We divide by $ \sqrt{d_k} $ for scaling the scores.  

${\text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right)}$

In [24]:
d_k = keys.shape[1]
attn_weights = torch.softmax(attn_scores / d_k**0.5, dim=-1)
print(attn_weights)

tensor([[0.1551, 0.2104, 0.2059, 0.1413, 0.1074, 0.1799],
        [0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
        [0.1503, 0.2256, 0.2192, 0.1315, 0.0914, 0.1819],
        [0.1591, 0.1994, 0.1962, 0.1477, 0.1206, 0.1769],
        [0.1610, 0.1949, 0.1923, 0.1501, 0.1265, 0.1752],
        [0.1557, 0.2092, 0.2048, 0.1419, 0.1089, 0.1794]])


**STEP 3:** Compute context vectors  

${\text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right)V}$

In [25]:
context_vec = attn_weights @ values
print(context_vec)

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]])
